# Function calling サンプル

現在、`Azure OpenAI Service` では使えない `OpenAI` が先行してリリースしているAPI

参考サイト
[GPT - OpenAI API function calling](https://platform.openai.com/docs/guides/gpt/function-calling)

In [26]:
import openai
import json
import os

In [27]:
os.environ["OPENAI_API_KEY"] = input("Enter your API key: ")
openai.api_key = os.getenv("OPENAI_API_KEY")

## 関数を定義

天気（ダミーデータ）を返す関数を定義

In [28]:
# Example dummy function hard coded to return the same weather
# In production, this could be your backend API or an external API
def get_current_weather(location, unit="fahrenheit"):
    """Get the current weather in a given location"""
    weather_info = {
        "location": location,
        "temperature": "72",
        "unit": unit,
        "forecast": ["sunny", "windy"],
    }
    return json.dumps(weather_info)

## Chat GPT を実行

`functions` に関数の情報を定義する。



In [29]:
def run_conversation():
    # Step 1: send the conversation and available functions to GPT
    messages = [{"role": "user", "content": "What's the weather like in Boston?"}]
    functions = [
        {
            "name": "get_current_weather",
            "description": "Get the current weather in a given location",
            "parameters": {
                "type": "object",
                "properties": {
                    "location": {
                        "type": "string",
                        "description": "The city and state, e.g. San Francisco, CA",
                    },
                    "unit": {"type": "string", "enum": ["celsius", "fahrenheit"]},
                },
                "required": ["location"],
            },
        }
    ]
    response = openai.ChatCompletion.create(
        model = "gpt-3.5-turbo-0613",
        messages = messages,
        functions = functions,
        function_call = "auto",  # auto is default, but we'll be explicit
    )
    response_message = response["choices"][0]["message"]

    print("--- first gpt response ---")
    print(response)

    # Step 2: check if GPT wanted to call a function
    if response_message.get("function_call"):
        # Step 3: call the function
        # Note: the JSON response may not always be valid; be sure to handle errors
        available_functions = {
            "get_current_weather": get_current_weather,
        }  # only one function in this example, but you can have multiple
        function_name = response_message["function_call"]["name"]
        fuction_to_call = available_functions[function_name]
        function_args = json.loads(response_message["function_call"]["arguments"])
        function_response = fuction_to_call(
            location = function_args.get("location"),
            unit = function_args.get("unit"),
        )

        print("--- function response ---")
        print(function_response)

        # Step 4: send the info on the function call and function response to GPT
        messages.append(response_message)  # extend conversation with assistant's reply
        messages.append(
            {
                "role": "function",
                "name": function_name,
                "content": function_response,
            }
        )  # extend conversation with function response
        second_response = openai.ChatCompletion.create(
            model="gpt-3.5-turbo-0613",
            messages=messages,
        )  # get a new response from GPT where it can see the function response
        
        print("--- second gpt response ---")
        print(second_response)
        
        return second_response


In [30]:
response = run_conversation()
print("--- final response ---")
print(response)

--- first gpt response ---
{
  "id": "chatcmpl-7c9U25YvbjaMOKW03rH4wF2nRxBb6",
  "object": "chat.completion",
  "created": 1689327274,
  "model": "gpt-3.5-turbo-0613",
  "choices": [
    {
      "index": 0,
      "message": {
        "role": "assistant",
        "content": null,
        "function_call": {
          "name": "get_current_weather",
          "arguments": "{\n  \"location\": \"Boston, MA\"\n}"
        }
      },
      "finish_reason": "function_call"
    }
  ],
  "usage": {
    "prompt_tokens": 82,
    "completion_tokens": 18,
    "total_tokens": 100
  }
}
--- function response ---
{"location": "Boston, MA", "temperature": "72", "unit": null, "forecast": ["sunny", "windy"]}
--- second gpt response ---
{
  "id": "chatcmpl-7c9U33I5iWvtJiNp0xieK59GdArvx",
  "object": "chat.completion",
  "created": 1689327275,
  "model": "gpt-3.5-turbo-0613",
  "choices": [
    {
      "index": 0,
      "message": {
        "role": "assistant",
        "content": "The current weather in Bosto